In [2]:
import pandas as pd

df = pd.read_csv("desafio1.csv")
df.head()

,RowNumber,id,sobrenome,pontuacao_credito,estado_residencia,genero,idade,nivel_estabilidade,saldo_conta,numero_produtos,possui_cartao_de_credito,membro_ativo
0,1,e7f44fcbd380d4cef7e6c232cc7e37895c3fd197,6d6e0aa1b9b413e442e2fb68df14b4fc3f91de50,619,SC,F,42,2,0.00,1,1,1
1,2,28dcb083ad90512da16b9430085c2cddb8ca5e12,48e1ad846796fa314f1b4a6702b83343eb5482c5,608,RS,F,41,1,83807.86,1,0,1
2,3,774bc378f787438c9c7594e536787d07a097a54b,f2b29d5d934de615812b697132e767dea0f1e9e2,502,SC,F,42,8,159660.80,3,1,0
3,4,043a71326f7096de155e7f0c559dc62b5e4b7239,85d5c9da7cddd8109ad32a6c348fe2bb10bf99f2,699,SC,F,39,1,0.00,2,0,0
4,5,37b3bb8ca243955fb3605ec7e1c2a4607cdb3b22,ddc89c837a6933639de75f28171057060bd322df,850,RS,F,43,2,125510.82,1,1,1


In [3]:
df.shape

(7000, 12)

In [123]:
state_mean = df.groupby('estado_residencia')['pontuacao_credito'].mean().reset_index()
state_median = df.groupby('estado_residencia')['pontuacao_credito'].median().reset_index()
state_mode = df.groupby('estado_residencia')['pontuacao_credito'].agg(lambda x:x.value_counts().index[0]).reset_index()
state_mode1 = df.groupby('estado_residencia')['pontuacao_credito'].apply(lambda x: x.mode())
state_mode2 = df.groupby('estado_residencia')['pontuacao_credito'].apply(lambda x: x.mode()[x.mode().index[-1]])
state_std = df.groupby('estado_residencia')['pontuacao_credito'].std().reset_index()

In [104]:
#state_mode_df = pd.DataFrame(state_mode)
#Certificando que a moda esta correta, pois o valor 850 ocorreu nos 3 estados
#df['pontuacao_credito'].loc[(df['estado_residencia'] == 'PR')].value_counts(sort=True)

In [206]:
df.estado_residencia.unique()

array(['SC', 'RS', 'PR'], dtype=object)

In [211]:
state_mean = f'{state_mean.pontuacao_credito: .3g}'
state_mean

TypeError: unsupported format string passed to Series.__format__

In [233]:
state_mode, state_mean, state_median, state_std
# escrever no json formato
#{state[0] : {"moda": 0.0, "mediana": 0.0, "media": 0.0, "desvio_padrao": 0.0}, state[1] : {}
states = df.estado_residencia.unique()

for state in states:
    media = state_mean.pontuacao_credito.loc[state_mean.estado_residencia == state]
    print(f"Estado: {state} - Media: {media}")
    
#state_mode[0] 
print(media)
print(state_mean)

Estado: SC - Media: 2    649.537653
Name: pontuacao_credito, dtype: float64
Estado: RS - Media: 1    651.105143
Name: pontuacao_credito, dtype: float64
Estado: PR - Media: 0    648.961294
Name: pontuacao_credito, dtype: float64
0    648.961294
Name: pontuacao_credito, dtype: float64
  estado_residencia  pontuacao_credito
0                PR         648.961294
1                RS         651.105143
2                SC         649.537653


In [159]:
print(state_mean.meanstate_mean.estado_residencia == state_mean.estado_residencia)

0    True
1    True
2    True
Name: estado_residencia, dtype: bool


In [164]:
#pd.concat([state_mean, state_median, state_mode, state_std], axis=1).drop_duplicates()
state_mean.merge(state_median, on = 'estado_residencia', how='left')

,estado_residencia,pontuacao_credito_x,pontuacao_credito_y
0,PR,648.961294,650
1,RS,651.105143,650
2,SC,649.537653,653


In [192]:
from functools import reduce
stats = [state_mode,state_median, state_mean,   state_std]
final_statistics = reduce(lambda left,right: pd.merge(left, right, on='estado_residencia'), stats )

In [194]:
final_statistics.columns=['estado_residencia', 'moda', 'mediana', 'media', 'desvio padrão']

In [232]:
final_statistics

,0,1,2
estado_residencia,PR,RS,SC
moda,850,850,850
mediana,650,650,653
media,648.961,651.105,649.538
desvio padrão,98.6072,95.1366,97.2335


In [231]:
#f strings - pesquisar
#dict_stats[stats[0]] =  { "moda" : stats[1],  "mediana" : stats[2], "media" : f'{stats[3]: .3f}', "desvio_padrao" : stats[4]}
dict_stats = {}
for index, stats in final_statistics.iterrows():
     dict_stats[stats[0]] =  { "moda" : stats[1],  "mediana" : stats[2], "media" : round(stats[3], 3), "desvio_padrao" : round(stats[4],3)}

#print(str(dict_stats).replace('\'','\"'))
        
with open("submission.json", "w") as file_submission:
    file_submission.write(str(dict_stats).replace('\'','\"'))